In [ ]:
import os
import re
import pickle
import numpy as np
from dotenv import dotenv_values
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from tqdm import tqdm
from evaluate import load
import pandas as pd

In [ ]:
# Load env file with API KEY using full path
config = dotenv_values(".env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_API_KEY"]

## Load test set

In [ ]:
from collections import defaultdict
with open("c2_data/column_gt.txt") as f:
    column_gt = defaultdict(dict)
    for line in f.readlines():
        table_name_col, concepts = line.strip('\n').split(',', 1)
        concepts = [concept.strip('"') for concept in concepts.split(",")]
        table_name, col_idx = table_name_col.strip('"').split(" ")
        column_gt[table_name][col_idx] = concepts
print(column_gt)

with open("c2_data/table_names.txt") as f:
    table_names = [line.strip('\n').strip('"') for line in f.readlines()]

labels = []
col_idxs = []
examples_top_rows = []
full_examples = []
for table_name in table_names:
    with open(f"c2_data/tables/{table_name}") as f:
        lines = list(f.readlines())
        lines = [line.strip("\n").replace("\t", " || ") for line in lines]
        columns_count = len(lines[0].split(" || "))
        columns_row_count = len(lines[1].split(" || "))
        if columns_row_count != columns_count:
            if columns_row_count < columns_count:
                for i in range(1, len(lines)):
                    lines[i] += " || " * (columns_count - columns_row_count)
            elif columns_row_count > columns_count:
                for i in range(1, len(lines)):
                    current = lines[i].split(" || ")
                    lines[i] = " || ".join(current[:columns_count])
        poss_labels = []
        table_col_idx = []
        if table_name in column_gt:
            for idx, concepts in column_gt[table_name].items():
                poss_labels.append(concepts[0])
                table_col_idx.append(idx)
        labels.append(poss_labels)
        col_idxs.append(table_col_idx)
        new_column_header = " || ".join([f"Column {i}" for i in range(columns_count)]) + "\n"
        examples_top_rows.append(new_column_header + "\n".join(lines[1:6]))
        full_examples.append(new_column_header + "\n".join(lines[1:]))

In [ ]:
table_id = 88
print(table_names[table_id])
print(examples_top_rows[table_id])
print(col_idxs[table_id])
print(labels[table_id])

## Choose prompt template: without or with instructions

In [ ]:
# Paper name: table
original_template = """

Answer the question based on the task below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Classify the columns of a given table with only one of the following classes that are separated with comma: description of event, description of restaurant, locality of address, postal code, region of address, country, price range, telephone, date, name of restaurant, payment accepted, day of week, review, organization, date and time, coordinate, name of event, event attendance mode, event status, currency, time, description of hotel, name of hotel, location feature, rating, fax number, email, photograph, name of music recording, music artist, name of album, duration.

Table: {input}

Class:

"""

# Paper name: table + instructions
original_inst_template = """

Answer the question based on the task and instructions below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Classify the columns of a given table with only one of the following classes that are separated with comma: description of event, description of restaurant, locality of address, postal code, region of address, country, price range, telephone, date, name of restaurant, payment accepted, day of week, review, organization, date and time, coordinate, name of event, event attendance mode, event status, currency, time, description of hotel, name of hotel, location feature, rating, fax number, email, photograph, name of music recording, music artist, name of album, duration.

Instructions: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, select a class that best represents the meaning of all cells in the column. 4. Answer with the selected class for each columns with the format Column1: class.

Table:
{input}

Class:

"""

In [ ]:
# New prompt
semantic_concept_template = """

Answer the question based on the task below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Suggest a semantic concept for each column of a given table. Answer with the semantic concept for each column with the format Column1: semantic concept.

Table: {input}

Semantic concepts:

"""

In [ ]:
# Paper name: table + instructions
inst_template = """

Answer the question based on the task and instructions below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Suggest a semantic concept for each column of a given table.

Instructions: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, suggest a semantic concept that best represents the meaning of all cells in the column. 4. Answer with the semantic concept for each column with the format Column1: semantic concept.

Table:
{input}

Semantic concepts:

"""

In [ ]:
# prompt to ask gpt to classify labels 
classify_label_template = """

Task: Classify the semantic concept {input} with only one of the following classes that are separated with comma: description of event, description of restaurant, locality of address, postal code, region of address, country, price range, telephone, date, name of restaurant, payment accepted, day of week, review, organization, date and time, coordinate, name of event, event attendance mode, event status, currency, time, description of hotel, name of hotel, location feature, rating, fax number, email, photograph, name of music recording, music artist, name of album, duration.

Class:

"""

In [ ]:
debug_template = """

Answer the question based on the task below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Suggest a semantic concept for each column of a given table. Answer with the semantic concept for each column with the format Column1: semantic concept.

Table: {input}

Semantic concepts:

"""

debug_template_top_5 = """

Answer the question based on the task below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Suggest 5 possible semantic concept for each column of a given table. Answer with the semantic concept for each column with the format Column1: possible semantic concept 1, possible semantic concept 2, possible semantic concept 3, possible semantic concept 4, possible semantic concept 5. 

Table: {input}

Semantic concepts:

"""

debug_template_album = """

Could this column be about music albums?

Table: {input}

"""

check_template = """

Critique whether these semantic concepts matches their respective columns in the given table and improve on them. If there is no further improvements to be made, just say 'It's good'.

Semantic concepts: {prev_output}

Table: {input}

Answer with the semantic concept for each column with the format Column1: semantic concept. 

Semantic concepts:

"""

check_template_best_of_5 = """

Critique whether these semantic concepts matches their respective columns in the given table and improve on them. Choose the best semantic concept for each column.

Semantic concepts: {prev_output}

Table: {input}

Answer with the semantic concept for each column with the format Column1: semantic concept. 

Semantic concepts:

"""

## Load LLM and run model

In [ ]:
gpt_3_turbo = ChatOpenAI(model_name='gpt-3.5-turbo-0301', temperature=0)
gpt_4 = ChatOpenAI(model_name='gpt-4-0613', temperature=0)

In [ ]:
prompt_type = "semantic_concept"
if prompt_type == "original_template":
    prompt = PromptTemplate(template=original_template, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)
elif prompt_type == "semantic_concept":
    prompt = PromptTemplate(template=semantic_concept_template, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)
elif prompt_type == "with_inst":
    prompt = PromptTemplate(template=inst_template, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)
elif prompt_type == "debug_template":
    prompt = PromptTemplate(template=debug_template, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)
elif prompt_type == "debug_template_top_5":
    prompt = PromptTemplate(template=debug_template_top_5, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)

In [ ]:
def convert_to_column_major(example: str) -> str:
    lines = example.split("\n")
    col_major = [col.strip() + ": " for col in lines[0].split("||")]
    for line in lines[1:]:
        for i, val in enumerate(line.split("||")):
            col_major[i] += val + ", "
    debug_eg = "\n".join(col_major) # Not needed to remove last row like when handling sotab
    return debug_eg

print(convert_to_column_major(examples_top_rows[5]))

In [ ]:
def save_preds(preds: list[str], file_name: str):
    #Save predictions in a file:
    with open(file_name,'wb') as f:
        pickle.dump(preds,f)

def load_preds(file_name: str):
    #Save predictions in a file:
    with open(file_name,'rb') as f:
        preds = pickle.load(f)
    return preds

In [ ]:
#Zero-shot prediction
preds_gpt_4 = []
for example in tqdm(examples_top_rows):
    preds_gpt_4.append(llm_chain_4.run({'input': example}))
save_preds(preds_gpt_4, "predictions_c2/gpt4-prompt-table-without-instructions.pkl")

In [ ]:
#Zero-shot prediction
preds_gpt_4 = []
for example in tqdm(examples_top_rows):
    preds_gpt_4.append(llm_chain_4.run({'input': convert_to_column_major(example)}))
save_preds(preds_gpt_4, "predictions_c2/gpt4-prompt-table-without-instructions-col-major.pkl")

In [ ]:
preds = load_preds("predictions_c2/gpt4-prompt-table-without-instructions.pkl")
print(preds)

In [ ]:
preds = load_preds("predictions_c2/gpt4-prompt-table-without-instructions-col-major.pkl")
print(preds)

In [ ]:
df = pd.DataFrame(data={"raw_prompt_output": preds})
df.to_csv("predictions/gpt4-prompt-table-without-instructions.csv")

## Evaluation

In [ ]:
bertscore = load("bertscore")

In [ ]:
def evaluation_new(preds, col_idxs, bert_threshold=0.85):
    ids, predictions, original_preds, highest_bertscores = [], [], [], []
    for j, table_preds in enumerate(tqdm(preds)):
        if "Semantic concepts:" in table_preds:
            table_preds = table_preds.split("Class:")[1]
        
        #Break predictions into either \n or ,
        if ":" in table_preds:
            separator = ":"
        elif "-" in table_preds:
            separator = "-"  
        else:
            separator = ","
            
        col_preds = table_preds.split(separator)
        i=0

        table_name = table_names[j]
        columns_to_eval = column_gt[table_name]
        for col_idx in col_idxs[j]:
            # print(idx, gt)
            gt = columns_to_eval[col_idx]
            ids.append(j)
            original_preds.append(table_preds)
            if int(col_idx) >= len(col_preds):
                predictions.append('-')
                highest_bertscores.append(0)
            else:
                pred = col_preds[int(col_idx)]
                # Remove break lines
                if "\n" in pred:
                    pred = pred.split('\n')[0].strip()
                # Remove commas
                if "," in pred:
                    pred = pred.split(",")[0].strip()
                # Remove paranthesis
                if '(' in pred:
                    pred = pred.split("(")[0].strip()
                #Remove points
                if '.' in pred:
                    pred = pred.split(".")[0].strip()
                # Lower-case prediction
                pred = pred.strip().lower()
                # print(pred, gt)
                bertscores = np.array(bertscore.compute(predictions=[pred]*len(gt), references=gt, lang="en")["f1"])
                highest_score = max(bertscores)
                highest_bertscores.append(highest_score)

                if highest_score > bert_threshold:
                    predictions.append(gt[0])
                else:
                    print(f"For test example {j} out of label space prediction: {pred}, true label: {gt}")
                    predictions.append('-')
            i+=1
            
    return ids, predictions, original_preds, highest_bertscores

ids, class_predictions, original_preds, highest_bertscores = evaluation_new(preds, col_idxs, bert_threshold=0.85)

In [ ]:
len(ids), len(class_predictions), len(original_preds), len(highest_bertscores)

In [ ]:
flattened_labels = [label for table in labels for label in table]
flattened_col_idxs = [idx for table in col_idxs for idx in table]
df = pd.DataFrame({"prompt_output_id": ids, 
                   "col_idx": flattened_col_idxs,
                   "label": flattened_labels, 
                   "original_pred": original_preds, 
                   "class_pred_using_bert": class_predictions, 
                   "highest_bertscore": highest_bertscores})
df.to_csv("predictions_c2/preds_gpt4_without_inst_col_major.csv")

### Calculate Precision, Recall, Macro-F1 and Micro-F1

In [ ]:
df = pd.read_csv("predictions_c2/preds_gpt4_without_inst_col_major.csv",index_col=0)
eval_labels, eval_preds = df["label"], df["class_pred_using_bert"]
df.head()

In [ ]:
print(len(df))
print(len(df[df["label"] == df["class_pred_using_bert"]]))
df['label_in_top_5'] = df[['label','top_5_preds']].apply(
    lambda row: row['label'] in row['top_5_preds'], axis=1
)
print(len(df[df["label_in_top_5"] == True]))

In [ ]:
def calculate_f1_scores(y_tests, y_preds):
    types = list(set(y_tests))
    types = types + ["-"]
    num_classes = len(types)
    
    y_tests = [types.index(y) for y in y_tests]
    y_preds = [types.index(y) for y in y_preds]
    
    #Confusion matrix
    cm = np.zeros(shape=(num_classes,num_classes))
    
    for i in range(len(y_tests)):
        cm[y_preds[i]][y_tests[i]] += 1
        
    report = {}
    
    for j in range(len(cm[0])):
        report[j] = {}
        report[j]['FN'] = 0
        report[j]['FP'] = 0
        report[j]['TP'] = cm[j][j]

        for i in range(len(cm)):
            if i != j:
                report[j]['FN'] += cm[i][j]
        for k in range(len(cm[0])):
            if k != j:
                report[j]['FP'] += cm[j][k]

        precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
        recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])
        f1 = 2*precision*recall / (precision + recall)
        
        if np.isnan(f1):
            f1 = 0
        if np.isnan(precision):
            f1 = 0
        if np.isnan(recall):
            f1 = 0

        report[j]['p'] =  precision
        report[j]['r'] =  recall
        report[j]['f1'] = f1
    
    all_fn = 0
    all_tp = 0
    all_fp = 0

    for r in report:
        if r != num_classes-1:
            all_fn += report[r]['FN']
            all_tp += report[r]['TP']
            all_fp += report[r]['FP']
        
    class_f1s = [ report[class_]['f1'] for class_ in report]
    class_p = [ 0 if np.isnan(report[class_]['p']) else report[class_]['p'] for class_ in report]
    class_r = [ 0 if np.isnan(report[class_]['r']) else report[class_]['r'] for class_ in report]
    macro_f1 = sum(class_f1s[:-1]) / (num_classes-1)
    
    p =  sum(class_p[:-1]) / (num_classes-1)
    r =  sum(class_r[:-1]) / (num_classes-1)
    micro_f1 = all_tp / ( all_tp + (1/2 * (all_fp + all_fn) )) 
    
    per_class_eval = {}
    for index, t in enumerate(types[:-1]):
        per_class_eval[t] = {"Precision":class_p[index], "Recall": class_r[index], "F1": class_f1s[index]}
    
    evaluation = {
        "Micro-F1": micro_f1,
        "Macro-F1": macro_f1,
        "Precision": p,
        "Recall": r
    }
    
    return evaluation, per_class_eval

In [ ]:
evaluation, per_class_eval = calculate_f1_scores(eval_labels, eval_preds)
print(evaluation)
print(per_class_eval)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, preds, average="micro")

## Error Analysis

In [ ]:
df = pd.read_csv("predictions/preds_gpt35_without_inst.csv",index_col=0)
df.head()

In [ ]:
error_df = df[df["lionel_annot"] != df["label"]]
error_df["table"] = error_df.apply(lambda row: examples[row["prompt_output_id"]], axis=1)
error_df["all_labels"] = error_df.apply(lambda row: test[row["prompt_output_id"]][2], axis=1)

In [ ]:
error_df.reset_index(inplace=True, drop=True)
error_df.head()

In [ ]:
for idx in range(len(error_df)):
    print("="*10)
    print(f"Table {error_df.loc[idx, 'prompt_output_id']}")
    print(error_df.loc[idx,"table"])
    print(error_df.loc[idx,"all_labels"])
    
    print(f"Ground truth: {error_df.loc[idx,'label']}")
    print(f"Raw output: {error_df.loc[idx,'parsed_col_pred']}")
    print(f"Annot: {error_df.loc[idx,'lionel_annot']}")

In [ ]:
# "-" means the model replied with out of label or with I don't know
errors = 0
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        errors += 1
        print(f"Predicted as {predictions[i]} when it was {labels[i]}")
errors